In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
import json

import keras
from keras.preprocessing import image

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
movies = pd.read_csv('archive/movies_metadata.csv')
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/Users/anshverma/opt/anaconda3/envs/python37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print("Shape: ", movies.shape)
print("Features: ", movies.columns)


Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [4]:
movies['overview'][1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [5]:
movies.head()['imdb_id']

0    tt0114709
1    tt0113497
2    tt0113228
3    tt0114885
4    tt0113041
Name: imdb_id, dtype: object

In [6]:
movies.head()['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [7]:
movies.head()['poster_path']

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

In [8]:
movies.head()['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [9]:
movies.head()['genres'][1]

"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"

In [10]:
# ARCHIVED FUNCTION
"""
def f(downloaded_image_dir, title, poster_path):
    if not os.path.exists(downloaded_image_dir):
        os.makedirs(downloaded_image_dir)
    
    # corrupted url
    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)
    filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'

    try:
        r = requests.get(imgUrl, stream=True)
        if r.status_code == 200:
            with open(downloaded_image_dir + '/' + filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print("Not Found: ", title, imgUrl)
        
    except:
        print("Error Dowloading: ", title, imgUrl)    
"""

'\ndef f(downloaded_image_dir, title, poster_path):\n    if not os.path.exists(downloaded_image_dir):\n        os.makedirs(downloaded_image_dir)\n    \n    # corrupted url\n    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)\n    filename = re.sub(r\'\\W+\', \' \', title).lower().strip().replace(" ", "-") + \'.jpg\'\n\n    try:\n        r = requests.get(imgUrl, stream=True)\n        if r.status_code == 200:\n            with open(downloaded_image_dir + \'/\' + filename, \'wb\') as f:\n                r.raw.decode_content = True\n                shutil.copyfileobj(r.raw, f)\n        else:\n            print("Not Found: ", title, imgUrl)\n        \n    except:\n        print("Error Dowloading: ", title, imgUrl)    \n'

In [91]:
# @Source: https://johannesbader.ch/blog/tutorial-download-posters-with-the-movie-database-api-in-python/"

import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()
    
def _download_images(titles, urls, path):
    """download all images in list 'urls' to 'path' """
    filePathList = []
    print(len(urls), len(titles))
    zipped = list(zip(titles, urls))
    for title, url in zipped:
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'
        filepath = os.path.join(path, filename)
        filePathList.append(filepath)
        
        if r.status_code == 200:
            with open(filepath,'wb') as w:
                w.write(r.content)
                
    return path, filePathList

def get_poster_url(imdbid):
    """ 
    given IMDB id, returns the corresponding url for
    poster jpg
    
    Args: 
        imdbid (str): IMDB id
    Return: 
        url (str): image url of poster image from 'themoviedb.org'
                of the maximum available size. 
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']   # sorted in ascending order
    
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    
    # retrieve image of max size
    max_size = max(sizes, key=size_str_to_int)
    
    posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    
    if len(posters) == 0:
        return None
    
    rel_path = posters[0]['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    
    return url

In [92]:
# RUN THIS FUNCTION TO DOWNLOAD JPG IMAGES
def tmdb_posters():
    """ download posters with imdb_id from dataset """
    
    if not os.path.exists('images_movies_genre'):
        os.makedirs('images_movies_genre')
    
    titles = []
    urls = []
    
    n = 0
    for index, row in movies.iterrows():
        title = str(row['title'])
        url = get_poster_url(row['imdb_id'])
        
        # skip iteration if poster cannot be located
        if not url:
            continue
            
        titles.append(title)
        urls.append(url)
        n += 1
        
        # SET LIMIT FOR NUMBER OF IMAGES TO DOWNLOAD
        #if n > 20:
        #    break

    return _download_images(titles, urls, 'images_movies_genre')

path, filePathList = tmdb_posters() 

ConnectionError: HTTPConnectionPool(host='api.themoviedb.org', port=80): Max retries exceeded with url: /3/configuration?api_key=6f2ceb0e0b459afe90f6e854fcd410eb (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb05750f2d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [13]:
print(path)

images_movies_genre


In [14]:
print(filePathList)

['images_movies_genre/toy-story.jpg', 'images_movies_genre/jumanji.jpg', 'images_movies_genre/grumpier-old-men.jpg', 'images_movies_genre/waiting-to-exhale.jpg', 'images_movies_genre/father-of-the-bride-part-ii.jpg', 'images_movies_genre/heat.jpg', 'images_movies_genre/sabrina.jpg', 'images_movies_genre/tom-and-huck.jpg', 'images_movies_genre/sudden-death.jpg', 'images_movies_genre/goldeneye.jpg', 'images_movies_genre/the-american-president.jpg', 'images_movies_genre/dracula-dead-and-loving-it.jpg', 'images_movies_genre/balto.jpg', 'images_movies_genre/nixon.jpg', 'images_movies_genre/cutthroat-island.jpg', 'images_movies_genre/casino.jpg', 'images_movies_genre/sense-and-sensibility.jpg', 'images_movies_genre/four-rooms.jpg', 'images_movies_genre/ace-ventura-when-nature-calls.jpg', 'images_movies_genre/money-train.jpg', 'images_movies_genre/get-shorty.jpg']


In [15]:
# TODO: jpg --> numpy array conversion

X = []

for path in tqdm(filePathList):
    img = image.load_img(path, target_size=(224, 224, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    
X = np.array(X)

100%|██████████| 21/21 [00:01<00:00, 19.50it/s]


In [16]:
image.array_to_img(X[0]).show()

In [17]:
#Padding
#reference: https://ai-pool.com/d/padding-images-with-numpy
def pad(img, h, w):
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint16)
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint16)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint16)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint16)
    return np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode='constant', constant_values=0))

In [18]:
#checking if padding works
img = image.load_img('images_movies_genre/toy-story.jpg', target_size=(224, 224, 3))
new_img_array=pad(X[0], 512, 512)
new_img=image.array_to_img(new_img_array)
new_img.show()

In [19]:
import ast
#reference-https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
def genre_list(str):
    arr=ast.literal_eval(str)
    genreList=[dict['name'] for dict in arr]
    return genreList

In [20]:
genre_list_collection=[]
for i in movies['genres']:
    genre_list_collection+=[genre_list(i)]
genre_list_collection[0]

['Animation', 'Comedy', 'Family']

In [21]:
#adding a column called genre_list which contains all the genres of the movies in a list
movies.insert(5,"genre_list", genre_list_collection, True)
print("Features: ", movies.columns)
movies['genre_list']

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object

In [22]:
"""Removing genres that are very sparse in dataset"""

""" archived code because unwanted genres already decided below
dictSet = {}
totGenres = 0
for movie in movies['genre_list']: 
    for genre in movie: 
        if dictSet.get(genre, 0): 
            dictSet[genre] += 1
        else: 
            dictSet[genre] = 1
        totGenres += 1

dictSetIter = dictSet.copy()
for genre in dictSetIter: 
    if not (dictSet[genre]/(totGenres) < 0.01):
        print(genre)
        dictSet.pop(genre)

print()

for movie in movies['genre_list']: 
    movieIter = movie.copy()
    for genre in movieIter: 
        if dictSet.get(genre, 0):
            movie.remove(genre)
            print(genre)
"""

" archived code because unwanted genres already decided below\ndictSet = {}\ntotGenres = 0\nfor movie in movies['genre_list']: \n    for genre in movie: \n        if dictSet.get(genre, 0): \n            dictSet[genre] += 1\n        else: \n            dictSet[genre] = 1\n        totGenres += 1\n\ndictSetIter = dictSet.copy()\nfor genre in dictSetIter: \n    if not (dictSet[genre]/(totGenres) < 0.01):\n        print(genre)\n        dictSet.pop(genre)\n\nprint()\n\nfor movie in movies['genre_list']: \n    movieIter = movie.copy()\n    for genre in movieIter: \n        if dictSet.get(genre, 0):\n            movie.remove(genre)\n            print(genre)\n"

In [23]:
#TOD0: Remove unwanted genres
invalid_genres = ['Aniplex', 'BROSTA TV', 'Carousel Productions', 'GoHands', 'Mardock Scramble Production Committee', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Sentai Filmworks', 'Telescene Film Group Productions', 'The Cartel', 'Vision View Entertainment', 'TV Movie', 'Foreign']
invalid_genres_dict = {}
for genre in invalid_genres: 
    invalid_genres_dict[genre] = 0
    
for movie in movies['genre_list']: 
    for genre in movie: 
        if invalid_genres_dict.get(genre, 0):
            movie.remove(genre)
            print(genre)            

In [24]:
all_genres = sum(genre_list_collection,[])
len(set(all_genres))

32

In [25]:
#One hot encoding the genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(movies['genre_list'].values)
genreLabelsList=mlb.transform(movies['genre_list'])
print(genreLabelsList.shape)
#TODO Add the one hot encoded genres to the movies dataframe
genre_labels=pd.DataFrame(genreLabelsList, columns=mlb.classes_)
print(genre_labels)

(45466, 32)
       Action  Adventure  Animation  Aniplex  BROSTA TV  Carousel Productions  \
0           0          0          1        0          0                     0   
1           0          1          0        0          0                     0   
2           0          0          0        0          0                     0   
3           0          0          0        0          0                     0   
4           0          0          0        0          0                     0   
...       ...        ...        ...      ...        ...                   ...   
45461       0          0          0        0          0                     0   
45462       0          0          0        0          0                     0   
45463       1          0          0        0          0                     0   
45464       0          0          0        0          0                     0   
45465       0          0          0        0          0                     0   

       Comedy  

In [26]:
genre_labels.to_csv('GenreLabels.csv', index=False)

In [27]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV',
       'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'], dtype=object)

In [28]:
movies['genre_list']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre_list, Length: 45466, dtype: object

In [29]:
overviews=pd.DataFrame(movies["overview"])
movies_with_overviews=pd.concat([overviews, genre_labels], axis=1)
movies_with_overviews.head()
#movies_with_overveiws=genre_labels
#movies_with_overviews["overview"]=movies["overview"]

,overview,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,"Led by Woody, Andy's toys live happily in his ...",0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,When siblings Judy and Peter discover an encha...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A family wedding reignites the ancient feud be...,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,"Cheated on, mistreated and stepped on, the wom...",0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Just when George Banks has recovered from his ...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
print("Shape: ", movies_with_overviews.shape)
print("Features: ", movies_with_overviews.columns)
movies_with_overviews.to_csv('MoviesWithOverviews.csv', index=False)

Shape:  (45466, 33)
Features:  Index(['overview', 'Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV',
       'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror',
       'Mardock Scramble Production Committee', 'Music', 'Mystery',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance',
       'Science Fiction', 'Sentai Filmworks', 'TV Movie',
       'Telescene Film Group Productions', 'The Cartel', 'Thriller',
       'Vision View Entertainment', 'War', 'Western'],
      dtype='object')


In [31]:
movies.shape

(45466, 25)

In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anshverma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
# Text cleaning
#reference: https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text): 
    #removing everything except letters and converting to lower case
    text = re.sub("[^a-zA-Z]"," ",str(text))
    text = ' '.join(text.split())
    text = text.lower() 
    return text

# Removing stopwords such as the, and, to, or etc
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [34]:
plot_summaries=[]
for text in movies['overview']:
    cleanText=clean_text(text)
    textWithoutStopwords=remove_stopwords(cleanText)
    plot_summaries+=[textWithoutStopwords]
movies.insert(11,"plot_summary", plot_summaries, True)
print("Features: ", movies.columns)

Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'genre_list', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'plot_summary', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count'],
      dtype='object')


In [35]:
movies["plot_summary"]

0        led woody andy toys live happily room andy bir...
1        siblings judy peter discover enchanted board g...
2        family wedding reignites ancient feud next doo...
3        cheated mistreated stepped women holding breat...
4        george banks recovered daughter wedding receiv...
                               ...                        
45461                             rising falling man woman
45462    artist struggles finish work storyline cult pl...
45463    one hits goes wrong professional assassin ends...
45464    small town live two brothers one minister one ...
45465    years decriminalisation homosexuality uk direc...
Name: plot_summary, Length: 45466, dtype: object

In [90]:
import torch
import torch.nn as nn
#from torchsummary import summary

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        print(block)
        super(ResNet, self).__init__()
        self.block = block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #ResNet layers
        self.layer1 = self.new_layer(num_blocks=layers[0], in_channels=64, out_channels=64, stride=1)
        self.layer2 = self.new_layer(num_blocks=layers[1], in_channels=64, out_channels=128, stride=2)
        self.layer3 = self.new_layer(num_blocks=layers[2], in_channels=128, out_channels=256, stride=2)
        self.layer4 = self.new_layer(num_blocks=layers[3], in_channels=256, out_channels=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        #print(x)
        print("input: ", x.shape)
        x = self.conv1(x)
        print("conv1: ", x.shape)
        x = self.bn1(x)
        #print("bn1: ", x.shape)
        x = self.relu(x)
        #print("relu: ", x.shape)
        x = self.maxpool(x)
        #print("maxpool: ", x.shape)

        x = self.layer1(x)
        #print("layer 1: ", x.shape)
        x = self.layer2(x)
        #print("layer 2: ", x.shape)
        x = self.layer3(x)
        #print("layer 3: ", x.shape)
        x = self.layer4(x)
        #print("layer 4: ", x.shape)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x
    
    def new_layer(self, num_blocks, in_channels, out_channels, stride):
        layers = []
        layers.append(self.block(in_channels, out_channels))
        intermediate_channels = out_channels
        for i in range(num_blocks):
            layers.append(self.block(intermediate_channels, out_channels))
           
        return nn.Sequential(*layers)

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Block, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.conv4 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn4 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.identity = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0)

    def forward(self, x):
        print(x)
        identity = x.clone()
        if self.in_channels != self.out_channels:
            identity = self.identity(identity)
        #print("id1", identity.shape)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        #Skip connection 1
        #print("id2", identity.shape, x.shape)
        x += identity
        identity = x.clone()
        #print("id3", identity.shape)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)

        #Skip connection 2
        x += identity

        x = self.relu(x)

        return x

def ResNet34():
    return ResNet(Block, [3, 4, 6, 3])

model = ResNet34()
#summary(model, input_size=(3, 224, 224), batch_size=1)

<class '__main__.Block'>


In [37]:
testDem = X[0].copy()

In [38]:
newArr = []
demensions = testDem.shape
count0 = 0
count1 = 0
count2 = 0

boolOne = False 
boolTwo = False
for demOne in testDem:
    count0 += 1
    for demTwo in demOne:
        if not boolOne:
            count1 += 1
        for demThree in demTwo: 
            if not boolTwo:
                count2 += 1
        boolTwo = True    
    boolOne = True
    #if (count0 < 224): 
    #    newArr.append(val)
    #count0 += 1
"""
for i in range(demensions[0]):
    count0 += 1
for j in range(demensions[1]): 
    count1 += 1
for q in range(demensions[2]): 
    count2 += 1
"""
print(count0)
print(count1)
print(count2)
#np.reshape(testDem, (3, 224, 224))

224
224
3


In [45]:
testDem = testDem.reshape((3, 224, 224))

In [47]:
print(testDem.shape)

(3, 224, 224)


In [72]:
newX = []
for i in range(len(X)):
    try:
        testDem = X[0].copy()
        testDem = testDem.reshape((3, 224, 224))
        newX.append(testDem)
        print("working")
    except:
        print("this one")
newX = np.array(newX)

working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working
working


In [73]:
for i in range(len(X)):
    print(X[i].shape)
    #np.reshape(X[i], (3, 224, 224))

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


In [74]:
for i in range(len(newX)):
    print(newX[i].shape)
    #np.reshape(X[i], (3, 224, 224))

(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)


In [77]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#need the data
#MNIST_train = MNIST("../data/mnist", train=True, download=True, transform = transforms.ToTensor())
#MNIST_test = MNIST("../data/mnist", train=False, download=True, transform = transforms.ToTensor())
train_dataloader = DataLoader(newX, batch_size=256, shuffle=True)
#test_dataloader = DataLoader(MNIST_test, batch_size=len(MNIST_test), shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [88]:
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    
    for batch in train_dataloader:
        X_batch, y_batch = batch[0], batch[1] #.view(-1, 784)

        #print(X_batch, y_batch)
        # THESE 5 LINES ARE IMPORTANT, UNDERSTAND WHAT IS HAPPENING HERE
        optimizer.zero_grad()
        predicted_batch = model(X_batch)
        #print(predicted_batch)
        loss = criterion(predicted_batch, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss
       
    print("Epoch {0}: {1}".format(epoch, total_loss))
    #if epoch%5 == 0 and epoch != 0:
    #    test_batch = next(iter(test_dataloader))
    #    X_test, y_test = test_batch[0].view(-1, 784), test_batch[1]
    #    predicted = model(X_test)
    #    test_acc = torch.sum(y_test == torch.argmax(predicted, dim=1), dtype=torch.double) / len(y_test)
    #    print("\tTest Accuracy {0}".format(test_acc))

input:  torch.Size([3, 224, 224])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 64 3 7 7, but got 3-dimensional input of size [3, 224, 224] instead